In [28]:
import sqlite3
import json
from datetime import datetime

In [29]:
timeframe = '2015-01'
sql_transaction = []
start_row = 0
cleanup = 1000000

In [30]:
connection = sqlite3.connect('{}.db'.format(timeframe))
c = connection.cursor()

In [31]:
def create_table():
    c.execute("CREATE TABLE IF NOT EXISTS parent_reply (parent_id TEXT PRIMARY KEY, comment_id TEXT UNIQUE, parent TEXT, comment TEXT, subreddit TEXT, unix INT, score INT)")

In [32]:
def format_data(data):
    data = data.replace('\n',' newlinechar ').replace('\r',' newlinechar ').replace('"',"'")
    return data

In [33]:
def transaction_bldr(sql):
    global sql_transaction
    sql_transaction.append(sql)
    if len(sql_transaction) > 1000:
        c.execute('BEGIN TRANSACTION')
        for s in sql_transaction:
            try:
                c.execute(s)
            except:
                pass
        connection.commit()
        sql_transaction = []

In [34]:
def acceptable(data):
    if len(data.split(' ')) > 1000 or len(data) < 1:
        return False
    elif len(data) > 32000:
        return False
    elif data == '[deleted]':
        return False
    elif data == '[removed]':
        return False
    else:
        return True

In [35]:
def sql_insert_replace_comment(commentid,parentid,parent,comment,subreddit,time,score):
    try:
        sql = """UPDATE parent_reply SET parent_id = ?, comment_id = ?, parent = ?, comment = ?, subreddit = ?, unix = ?, score = ? WHERE parent_id =?;""".format(parentid, commentid, parent, comment, subreddit, int(time), score, parentid)
        transaction_bldr(sql)
    except Exception as e:
        print('s0 insertion',str(e))

In [36]:
def sql_insert_has_parent(commentid,parentid,parent,comment,subreddit,time,score):
    try:
        sql = """INSERT INTO parent_reply (parent_id, comment_id, parent, comment, subreddit, unix, score) VALUES ("{}","{}","{}","{}","{}",{},{});""".format(parentid, commentid, parent, comment, subreddit, int(time), score)
        transaction_bldr(sql)
    except Exception as e:
        print('s0 insertion',str(e))

In [37]:
def sql_insert_no_parent(commentid,parentid,comment,subreddit,time,score):
    try:
        sql = """INSERT INTO parent_reply (parent_id, comment_id, comment, subreddit, unix, score) VALUES ("{}","{}","{}","{}",{},{});""".format(parentid, commentid, comment, subreddit, int(time), score)
        transaction_bldr(sql)
    except Exception as e:
        print('s0 insertion',str(e))

In [38]:
def find_parent(pid):
    try:
        sql = "SELECT comment FROM parent_reply WHERE comment_id = '{}' LIMIT 1".format(pid)
        c.execute(sql)
        result = c.fetchone()
        if result != None:
            return result[0]
        else: return False
    except Exception as e:
        #print(str(e))
        return False

In [39]:
def find_existing_score(pid):
    try:
        sql = "SELECT score FROM parent_reply WHERE parent_id = '{}' LIMIT 1".format(pid)
        c.execute(sql)
        result = c.fetchone()
        if result != None:
            return result[0]
        else: return False
    except Exception as e:
        #print(str(e))
        return False

In [40]:
if __name__ == '__main__':
    create_table()
    row_counter = 0
    paired_rows = 0

    with open('E:/hst_sync_2017/python/text_learning_project/chatbot/deep_learning_chatbot/{}/RC_{}'.format(timeframe.split('-')[0],timeframe), buffering=1000) as f:
    #with open('{}/RC_{}'.format(timeframe.split('-')[0],timeframe), buffering=1000) as f:
        for row in f:
            #print(row)
            #time.sleep(555)
            row_counter += 1

            if row_counter > start_row:
                try:
                    row = json.loads(row)
                    parent_id = row['parent_id'].split('_')[1]
                    body = format_data(row['body'])
                    created_utc = row['created_utc']
                    score = row['score']
                    
                    comment_id = row['id']
                    
                    subreddit = row['subreddit']
                    parent_data = find_parent(parent_id)
                    
                    existing_comment_score = find_existing_score(parent_id)
                    if existing_comment_score:
                        if score > existing_comment_score:
                            if acceptable(body):
                                sql_insert_replace_comment(comment_id,parent_id,parent_data,body,subreddit,created_utc,score)
                                
                    else:
                        if acceptable(body):
                            if parent_data:
                                if score >= 2:
                                    sql_insert_has_parent(comment_id,parent_id,parent_data,body,subreddit,created_utc,score)
                                    paired_rows += 1
                            else:
                                sql_insert_no_parent(comment_id,parent_id,body,subreddit,created_utc,score)
                except Exception as e:
                    print(str(e))
                            
            if row_counter % 100000 == 0:
                print('Total Rows Read: {}, Paired Rows: {}, Time: {}'.format(row_counter, paired_rows, str(datetime.now())))

            if row_counter > start_row:
                if row_counter % cleanup == 0:
                    print("Cleanin up!")
                    sql = "DELETE FROM parent_reply WHERE parent IS NULL"
                    c.execute(sql)
                    connection.commit()
                    c.execute("VACUUM")
                    connection.commit()

Total Rows Read: 100000, Paired Rows: 6105, Time: 2018-01-08 17:43:00.992415
Total Rows Read: 200000, Paired Rows: 13837, Time: 2018-01-08 17:43:41.466730
Total Rows Read: 300000, Paired Rows: 22238, Time: 2018-01-08 17:44:30.408530
Total Rows Read: 400000, Paired Rows: 30681, Time: 2018-01-08 17:45:29.848929
Total Rows Read: 500000, Paired Rows: 38915, Time: 2018-01-08 17:46:34.577632
Total Rows Read: 600000, Paired Rows: 46399, Time: 2018-01-08 17:47:41.447456
Total Rows Read: 700000, Paired Rows: 54145, Time: 2018-01-08 17:48:49.522350
Total Rows Read: 800000, Paired Rows: 62623, Time: 2018-01-08 17:49:57.875260
Total Rows Read: 900000, Paired Rows: 71345, Time: 2018-01-08 17:50:58.762742
Total Rows Read: 1000000, Paired Rows: 79923, Time: 2018-01-08 17:51:57.026075
Cleanin up!
Total Rows Read: 1100000, Paired Rows: 87076, Time: 2018-01-08 17:52:38.713459
Total Rows Read: 1200000, Paired Rows: 95201, Time: 2018-01-08 17:53:22.521965
Total Rows Read: 1300000, Paired Rows: 103703, Tim